For the theoretical background regarding these techniques , pleases refer to theoretical discussion on linear models. 

Now we'll discuss a case study solution using multiple linear regression, and regularised linear regresison [Ridge and Lasso] . We'll also look at hyper parameter tuning for regularised regression.

A little background on the case study. This data belongs to a loan aggregator agency which connects loan applications to different financial institutions in attempt to get the best interest rate. They want to now utilise past data to predict interest rate given by any financial institute just by looking at loan application characteristics.

To achieve that , they have decided to do a POC with a data from a particular financial institution. The data is given in the file "loans data.csv". Lets begin: 

In [33]:
data_file=r'F:/Data Science/Python/Data/loans data.csv'

import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge
import numpy as np
from sklearn.model_selection import KFold
%matplotlib inline

ld=pd.read_csv(data_file)


In [ ]:
ld.head()

You can see that variable Interest.Rate and Debt.To.Income.Ratio contain "%" sign in their values and because of which they have come as character columns in the data. Lets remove these percentages first.

In [ ]:
for col in ["Interest.Rate","Debt.To.Income.Ratio"]:
    ld[col]=ld[col].astype("str")
    ld[col]=[x.replace("%","") for x in ld[col]]



In [ ]:
ld.dtypes

We can see that many columns which should have really been numbers have been imported as character columns , probably because some characters values in those columns in the files. We'll convert all such columns to numbers .

In [ ]:
for col in ["Amount.Requested","Amount.Funded.By.Investors","Open.CREDIT.Lines","Revolving.CREDIT.Balance",
           "Inquiries.in.the.Last.6.Months","Interest.Rate","Debt.To.Income.Ratio"]:
    ld[col]=pd.to_numeric(ld[col],errors="coerce")

In [ ]:
ld.dtypes

Next we will make dummy variables for remaining categorical variables

In [ ]:
ld["Loan.Length"].value_counts()

Function get_dummies creates dummy variables for all the categories present in the categorical variable. Result is a dataframe, we can then choose and drop the dummies that we want to drop and attach the ones selected back to our original data.

In [ ]:
ll_dummies=pd.get_dummies(ld["Loan.Length"])

In [ ]:
ll_dummies.head()

We'll add dummy variable for "36 months" to our data and ignore the rest two. 

In [ ]:
ld["LL_36"]=ll_dummies["36 months"]

Now that we'are done with dataframe ll_dummies , we can drop it. Below we demonstrate a general way of removing variables from notebook environment.

In [ ]:
%reset_selective ll_dummies

To know what all variables are in the environment, you can use function "who"

In [ ]:
who

Now that we have created dummies for Loan.Length, we need to remove this from the dataframe.

In [ ]:
ld=ld.drop('Loan.Length',axis=1)

In [ ]:
ld.dtypes

Next we examine variable "Loan.Purpose".

In [ ]:
ld["Loan.Purpose"].value_counts()

There are 14 categories in that variable, we can either make 13 dummies or we can club few of these categories together and reduce the number of effective categories and then make dummy variables for those.

It makes sense to club those categories which behave similarly in terms of their effect on response. Or in other words , we can club those categories for which average interest rates are similar in the data.

In [ ]:
round(ld.groupby("Loan.Purpose")["Interest.Rate"].mean())

We can see from the table above that there are 4 effective categoris in the data. Lets club them

In [ ]:
for i in range(len(ld.index)):
    if ld["Loan.Purpose"][i] in ["car","educational","major_purchase"]:
        ld.loc[i,"Loan.Purpose"]="cem"
    if ld["Loan.Purpose"][i] in ["home_improvement","medical","vacation","wedding"]:
        ld.loc[i,"Loan.Purpose"]="hmvw"
    if ld["Loan.Purpose"][i] in ["credit_card","house","other","small_business"]:
        ld.loc[i,"Loan.Purpose"]="chos"
    if ld["Loan.Purpose"][i] in ["debt_consolidation","moving"]:
        ld.loc[i,"Loan.Purpose"]="dm"

Now we make dummies for this variable

In [ ]:
lp_dummies=pd.get_dummies(ld["Loan.Purpose"],prefix="LP")

In [ ]:
lp_dummies.head()

We'll add this data to original data. And then drop original variable "Loan.Purpose" and one of the dummies "LP_renewable_energy"

In [ ]:
ld=pd.concat([ld,lp_dummies],1)
ld=ld.drop(["Loan.Purpose","LP_renewable_energy"],1)

In [ ]:
ld.dtypes

Next we look at variable "State".

In [ ]:
ld["State"].nunique()

There are too many unique values. Although its not a legit reason to drop a variable. But we'll ignore this in this discussion any way in order to reduce amount of data prep that we are doing here. You can try including it in the model and see if the performance improves.

In [ ]:
ld=ld.drop(["State"],1)

Next we take care of variable Home.Ownership.

In [ ]:
ld["Home.Ownership"].value_counts()

In [ ]:
ld["ho_mort"]=np.where(ld["Home.Ownership"]=="MORTGAGE",1,0)
ld["ho_rent"]=np.where(ld["Home.Ownership"]=="RENT",1,0)
ld=ld.drop(["Home.Ownership"],1)

We have simply ignored values OTHER and NONE , and considered that there are only 3 categories and created only two dummies . We did this because of very low frequencies of OTHER and NONE

In [ ]:
ld["FICO.Range"].head()

If you look at first few values of variable FICO.Range , you can see that we can convert it to numeric by taking average of the range given. To do that first we need to split the column with "-", so that we can have both end of ranges in separate columns and then we can simply average them.
Lets first split.

In [ ]:
ld['f1'], ld['f2'] = zip(*ld['FICO.Range'].apply(lambda x: x.split('-', 1)))

Now we create new variable "fico" by averaging f1 and f2. And then we'll drop the original variable FICO.Range and f1,f2.

In [ ]:
ld["fico"]=0.5*(pd.to_numeric(ld["f1"])+pd.to_numeric(ld["f2"]))

ld=ld.drop(["FICO.Range","f1","f2"],1)

Next we look at variavle Employment.Length. You'll see that we can convert that to number as well.

In [ ]:
ld["Employment.Length"].value_counts()

In [ ]:
ld["Employment.Length"]=ld["Employment.Length"].astype("str")
ld["Employment.Length"]=[x.replace("years","") for x in ld["Employment.Length"]]
ld["Employment.Length"]=[x.replace("year","") for x in ld["Employment.Length"]]

We can convert everything else to numbers , but "n/a" are a problem. We can look at average interest across all values of Employment.Length and then replace "n/a" with value which has closet average response.

In [ ]:
round(ld.groupby("Employment.Length")["Interest.Rate"].mean(),2)

As you can see "n/a" is similar to "< 1".

In [ ]:
ld["Employment.Length"]=[x.replace("n/a","< 1") for x in ld["Employment.Length"]]
ld["Employment.Length"]=[x.replace("10+","10") for x in ld["Employment.Length"]]
ld["Employment.Length"]=[x.replace("< 1","0") for x in ld["Employment.Length"]]
ld["Employment.Length"]=pd.to_numeric(ld["Employment.Length"],errors="coerce")

In [ ]:
ld.dtypes

Now we have all the variables as numbers. After dropping observations with missing values , we can proceed to build oru model.

In [ ]:
ld.shape

In [ ]:
ld.dropna(axis=0,inplace=True)

In [ ]:
ld.shape

We now split our data into two random parts . One to build model on , Another to test its performance. Option "random_state" is used to make our random operation reproducible.

In [ ]:
ld_train, ld_test = train_test_split(ld, test_size = 0.2,random_state=2)

In [ ]:
lm=LinearRegression()

Above line creates and object of class LinearRegression named lm. We can use this object to access all functions realted to LinearRegression.

Now we'll separate predictors and response for both the datasets . We'll also drop ID from predictor's list because it doesnt make sense to include an ID variable in the model. Variable "Amount.Funded.By.Investors" will also be dropped because it wont be available until the loan has been processed. We can use only those variables which are present at the point of the business process where we want to apply our model.

In [ ]:
x_train=ld_train.drop(["Interest.Rate","ID","Amount.Funded.By.Investors"],1)
y_train=ld_train["Interest.Rate"]
x_test=ld_test.drop(["Interest.Rate","ID","Amount.Funded.By.Investors"],1)
y_test=ld_test["Interest.Rate"]

Now we can fit our model using lm the LinearRegression object that we created earlier

In [ ]:
lm.fit(x_train,y_train)

Next we predict resposne on our test data , calculate errors on those prediction and then rmse for those residuals. That is the measure of performance on the test data. We can use this measure to compare other models that we'll build.

In [ ]:
p_test=lm.predict(x_test)

residual=p_test-y_test

rmse_lm=np.sqrt(np.dot(residual,residual)/len(p_test))

rmse_lm

In [ ]:
y_test["Interest.Rate"]

We can use this to compare our linear regression model with other techniques and evenutall pick the one with least error .

Next we show how to extract coefficient produced by our model

In [ ]:
coefs=lm.coef_

features=x_train.columns

list(zip(features,coefs))

We can see that linear regression has produced coefficients for all variables. If you recall our theoretical discussion, we need to penalise coefficient for the variables which are not really contributing well to our resposne and might be causing overfitting of the model. Among the regularised technique we'll first look at Ridge regression.

Since penalty in ridge regression is a hyperparameter , we'd look at multiple values of it and choose the best one through 10 fold cross validation.

In [ ]:
# Finding best value of penalty weight with cross validation for ridge regression
alphas=np.linspace(2,10,100)
# We need to reset index for cross validation to work without hitch
x_train.reset_index(drop=True,inplace=True)
y_train.reset_index(drop=True,inplace=True)

In [ ]:
kf = KFold(n_splits=10)
for train, test in kf,split(x_train):
    print(train)
    print("-----")
    print(test)

In [ ]:
rmse_list=[]
for a in alphas:
    ridge = Ridge(fit_intercept=True, alpha=a)

    # computing average RMSE across 10-fold cross validation
   # kf = KFold(len(x_train), n_folds=10)
    kf = KFold(n_splits=10)
    xval_err = 0
    for train, test in kf.split(x_train):
        ridge.fit(x_train.iloc[train], y_train.iloc[train])
        p = ridge.predict(x_train.loc[test])
        err = p - y_train[test]
        xval_err += np.dot(err,err)
    rmse_10cv = np.sqrt(xval_err/len(x_train))
    # uncomment below to print rmse values for individidual alphas
#     print('{:.3f}\t {:.6f}\t '.format(a,rmse_10cv))
    rmse_list.extend([rmse_10cv])
best_alpha=alphas[rmse_list==min(rmse_list)]
print('Alpha with min 10cv error is : ',best_alpha )

best value of alpha  might be slightly different across different runs because of random nature of cross validation. So dont worry if you determine a different value of best alpha.

Next we fit Ridge Regression on the entire train data with best value of alpha we just determined.


In [ ]:

ridge=Ridge(fit_intercept=True,alpha=best_alpha)

ridge.fit(x_train,y_train)

p_test=ridge.predict(x_test)

residual=p_test-y_test

rmse_ridge=np.sqrt(np.dot(residual,residual)/len(p_test))

rmse_ridge

In [ ]:
list(zip(x_train.columns,ridge.coef_))

In [ ]:
list(zip(features,coefs))

You can see that ridge regression though, shrinks the coefficients but never makes them exactly zero, essentially never reduce our model size. Next we look at lasso Regression.

In [ ]:
alphas=np.linspace(0.0001,1,100)
rmse_list=[]
for a in alphas:
    lasso = Lasso(fit_intercept=True, alpha=a,max_iter=10000)

    # computing RMSE using 10-fold cross validation
   # kf = KFold(len(x_train), n_folds=10)
    kf = KFold(n_splits=10)
    xval_err = 0
    for train, test in kf.split(x_train):
        lasso.fit(x_train.loc[train], y_train[train])
        p =lasso.predict(x_train.loc[test])
        err = p - y_train[test]
        xval_err += np.dot(err,err)
    rmse_10cv = np.sqrt(xval_err/len(x_train))
    rmse_list.extend([rmse_10cv])
    # Uncomment below to print rmse values of individual alphas
   # print('{:.3f}\t {:.4f}\t '.format(a,rmse_10cv))
best_alpha=alphas[rmse_list==min(rmse_list)]
print('Alpha with min 10cv error is : ',best_alpha )

In [ ]:
lasso=Lasso(fit_intercept=True,alpha=best_alpha)

lasso.fit(x_train,y_train)

p_test=lasso.predict(x_test)

residual=p_test-y_test

rmse_lasso=np.sqrt(np.dot(residual,residual)/len(p_test))

rmse_lasso

In [ ]:
list(zip(x_train.columns,lasso.coef_))

We can see that lasso regression, not only improves performance on the data slightly , but also makes size of the model smaller by making many coefficents exactly zero, thus excluding them from our model.

### Logistic Model for Binary Classification

A retail banking institution is going to float a stock trading facility for their existing customer. Since this kind of facitlity is nothing new , company knows that they will have to incetivise their customers for adopting their offerings. One way to incetiwise is to offer discounts on the commision for trading transactions.

One issue with that is that only about 10% of the customers do enought trades for earnings after discounts to be profitable. Company wants to figure out, which are those 10% customer so that it can selectively offer them discount. there is no magic way to figure that out. So company rolled out this service to about 10000+ of their customers and observed their trading behaviour for 6 months and after that they labelled them into two revenue.grids 1 and 2.
using this data, now they want us to build a classification model which can be used to classify their remaining customers into these revenue grids.



#### Logistic Regression from Scikit Learn 

Logistic Regression in scikit learn already contains penalties. l1 and l2 [Read as L-one & L-two] penalties . l1 penalty is same as lasso penalty where as l2 is same as ridge penalty. parameter C for logistic regression function is the hyperparameter for penalty . However it works in inverse fashion, i.e. if C takes smalle , it means higher penalty. 

For the case that we have discussed here , we have discussed l1 penalty with value of C as 1. We have left following things for you to try on your own.

* model with l2 penalty
* Finding optimal value of hyperparameter C with cross-validation for both the penalties

You will find these in the practice exercise as well

you can use auc value obtained from function roc_auc_score to select best value for the hyperparameter. Higher the auc, better is the model . If you dont recall this, please go back to the theoretical reading material.

Lets beging our model building process.


# Confusion Matrix

In [ ]:
Fb = (1 + B^2) precision * recall / (B^2) precision + recall

IF beta = 1

Fi score 2*P*R/P+R
------------------------
beta = 0 

P = 1*P*R/R

------------------------

KS Score = (TP/P) - (FP/N)

min = 0 
max = 100

In [3]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [4]:
data_file=r'F:/Data Science/Python/Data/Existing Base.csv'
bd=pd.read_csv(data_file)

In [ ]:
bd.head()

In [ ]:
bd["children"].value_counts()

It seems we can directly convert this to numeric.

In [5]:
bd.loc[bd["children"]=="Zero","children"]="0"
bd.loc[bd["children"]=="4+","children"]="4"
bd["children"]=pd.to_numeric(bd["children"],errors="coerce")

In [ ]:
bd["Revenue Grid"].value_counts()

In [6]:
bd["y"]=np.where(bd["Revenue Grid"]==2,0,1)
bd=bd.drop(["Revenue Grid"],1)

For variable , age_band if we treat it as categorical variable , we can combine its categories by looking average response rate across its categories.

In [16]:
round(bd.groupby("age_band")["y"].mean(),2)

KeyError: 'age_band'

In [8]:
for i in range(len(bd)):
    if bd["age_band"][i] in ["71+","65-70","51-55","45-50"]:
        bd.loc[i,"age_band"]="ab_10"
    if bd["age_band"][i] in ["55-60","41-45","31-35","22-25","26-30"]:
        bd.loc[i,"age_band"]="ab_11"
    if bd["age_band"][i]=="36-40":
        bd.loc[i,"age_band"]="ab_13"
    if bd["age_band"][i]=="18-21":
        bd.loc[i,"age_band"]="ab_17"
    if bd["age_band"][i]=="61-65":
        bd.loc[i,"age_band"]="ab_9"
ab_dummies=pd.get_dummies(bd["age_band"])
ab_dummies.head()

,Unknown,ab_10,ab_11,ab_13,ab_17,ab_9
0,0,1,0,0,0,0
1,0,0,1,0,0,0
2,0,0,1,0,0,0
3,0,0,0,0,1,0
4,0,1,0,0,0,0


We will add it back to the dataset, dropping the dummy for "Unknown".

In [9]:
bd=pd.concat([bd,ab_dummies],1)
bd=bd.drop(["age_band","Unknown"],1)

In [ ]:
bd["status"].value_counts()

In [10]:
bd["st_partner"]=np.where(bd["status"]=="Partner",1,0)
bd["st_singleNm"]=np.where(bd["status"]=="Single/Never Married",1,0)
bd["st_divSep"]=np.where(bd["status"]=="Divorced/Separated",1,0)
bd=bd.drop(["status"],1)

In [17]:
round(bd.groupby("occupation")["y"].mean(),2)

occupation
Housewife    0.09
oc_10        0.10
oc_11        0.11
oc_12        0.12
Name: y, dtype: float64

In [11]:
for i in range(len(bd)):
    if bd["occupation"][i] in ["Unknown","Student","Secretarial/Admin","Other","Manual Worker"]:
        bd.loc[i,"occupation"]="oc_11"
    if bd["occupation"][i] in ["Professional","Business Manager"]:
        bd.loc[i,"occupation"]="oc_12"
    if bd["occupation"][i]=="Retired":
        bd.loc[i,"occupation"]="oc_10"
oc_dummies=pd.get_dummies(bd["occupation"])
oc_dummies.head()

,Housewife,oc_10,oc_11,oc_12
0,0,0,1,0
1,0,1,0,0
2,0,0,0,1
3,0,0,0,1
4,0,0,0,1


In [18]:
bd=pd.concat([bd,oc_dummies],1)

bd=bd.drop(["occupation","Housewife"],1)

In [19]:
round(bd.groupby("occupation_partner")["y"].mean(),2)

KeyError: 'occupation_partner'

In [20]:
bd["ocp_10"]=0
bd["ocp_12"]=0
for i in range(len(bd)):
    if bd["occupation_partner"][i] in ["Unknown","Retired","Other"]:
        bd.loc[i,"ocp_10"]=1
    if bd["occupation_partner"][i] in ["Student","Secretarial/Admin"]:
        bd.loc[i,"ocp_12"]=1
        
bd=bd.drop(["occupation_partner","TVarea","post_code","post_area","region"],1)

KeyError: 'occupation_partner'

You can see that we have also dropped variables TVarea, region, post_code, post_area. If you look at number of unique values taken by post_area and post_code , you'll realise why decided to drop them. TVarea and region on the other hand we have left for you to make use of and see if using them improves your model.

In [15]:
bd["home_status"].value_counts()

Own Home                9413
Rent from Council/HA     322
Rent Privately           261
Live in Parental Hom     109
Unclassified              50
Name: home_status, dtype: int64

In [21]:
bd["hs_own"]=np.where(bd["home_status"]=="Own Home",1,0)
del bd["home_status"]

Notice that we used an alternate syntax for dropping a column here. You can use that too if you like this syntax better.

In [22]:
bd["gender"].value_counts()

Female     7634
Male       2486
Unknown      35
Name: gender, dtype: int64

In [24]:
bd["gender_f"]=np.where(bd["gender"]=="Female",1,0)
del bd["gender"]

KeyError: 'gender'

In [25]:
bd["self_employed"].value_counts()

No     9436
Yes     719
Name: self_employed, dtype: int64

In [26]:
bd["semp_yes"]=np.where(bd["self_employed"]=="Yes",1,0)
del bd["self_employed"]

In [27]:
bd["self_employed_partner"].value_counts()

No     9026
Yes    1129
Name: self_employed_partner, dtype: int64

In [28]:
bd["semp_part_yes"]=np.where(bd["self_employed_partner"]=="Yes",1,0)
del bd["self_employed_partner"]

In [29]:
bd["family_income"].value_counts()

>=35,000             2517
<27,500, >=25,000    1227
<30,000, >=27,500     994
<25,000, >=22,500     833
<20,000, >=17,500     683
<12,500, >=10,000     677
<17,500, >=15,000     634
<15,000, >=12,500     629
<22,500, >=20,000     590
<10,000, >= 8,000     563
< 8,000, >= 4,000     402
< 4,000               278
Unknown               128
Name: family_income, dtype: int64

We can convert this to number as average of the range once we have figured out what to do with category "Unknown".

In [30]:
round(bd.groupby("family_income")["y"].mean(),4)

family_income
< 4,000              0.0755
< 8,000, >= 4,000    0.0796
<10,000, >= 8,000    0.1066
<12,500, >=10,000    0.1019
<15,000, >=12,500    0.1113
<17,500, >=15,000    0.1230
<20,000, >=17,500    0.1113
<22,500, >=20,000    0.1186
<25,000, >=22,500    0.1032
<27,500, >=25,000    0.0970
<30,000, >=27,500    0.1157
>=35,000             0.1116
Unknown              0.0703
Name: y, dtype: float64

In [31]:
bd["fi"]=4 # by doing this , we have essentially clubbed <4000 and Unknown values . How?
bd.loc[bd["family_income"]=="< 8,000, >= 4,000","fi"]=6
bd.loc[bd["family_income"]=="<10,000, >= 8,000","fi"]=9
bd.loc[bd["family_income"]=="<12,500, >=10,000","fi"]=11.25
bd.loc[bd["family_income"]=="<15,000, >=12,500","fi"]=13.75
bd.loc[bd["family_income"]=="<17,500, >=15,000","fi"]=16.25
bd.loc[bd["family_income"]=="<20,000, >=17,500","fi"]=18.75
bd.loc[bd["family_income"]=="<22,500, >=20,000","fi"]=21.25
bd.loc[bd["family_income"]=="<25,000, >=22,500","fi"]=23.75
bd.loc[bd["family_income"]=="<27,500, >=25,000","fi"]=26.25
bd.loc[bd["family_income"]=="<30,000, >=27,500","fi"]=28.75
bd.loc[bd["family_income"]==">=35,000","fi"]=35
bd=bd.drop(["family_income"],1)

In [ ]:
bd.dtypes

Now that the entire data is of numeric type, lets beging our modelling process after removing nas from the data.

In [34]:
bd.dropna(axis=0,inplace=True)
bd_train, bd_test = train_test_split(bd, test_size = 0.2,random_state=2)

In [35]:
x_train=bd_train.drop(["y","REF_NO"],1)
y_train=bd_train["y"]
x_test=bd_test.drop(["y","REF_NO"],1)
y_test=bd_test["y"]

In [39]:
bd.dtypes

REF_NO                               int64
children                             int64
year_last_moved                      int64
Average Credit Card Transaction    float64
Balance Transfer                   float64
Term Deposit                       float64
Life Insurance                     float64
Medical Insurance                  float64
Average A/C Balance                float64
Personal Loan                      float64
Investment in Mutual Fund          float64
Investment Tax Saving Bond         float64
Home Loan                          float64
Online Purchase Amount             float64
Investment in Commudity            float64
Investment in Equity               float64
Investment in Derivative           float64
Portfolio Balance                  float64
y                                    int32
ab_10                                uint8
ab_11                                uint8
ab_13                                uint8
ab_17                                uint8
ab_9       

In [36]:
logr=LogisticRegression(penalty="l1",class_weight="balanced",random_state=2)

In [38]:
logr.fit(x_train,y_train)

C:\Users\Tejas\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='warn', n_jobs=None, penalty='l1',
                   random_state=2, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [40]:
# score model performance on the test data
roc_auc_score(y_test,logr.predict(x_test))

0.8990384615384616

To arrive at the eventual 1,0 prediction , we need to find some way [some cutoff ] to convert predicted probabilites into two classes. Lets first get the probabilities out.

In [41]:
prob_score=pd.Series(list(zip(*logr.predict_proba(x_train)))[1])

In [42]:
prob_score

0       0.040266
1       0.040121
2       0.014102
3       0.243890
4       0.040045
5       0.181230
6       0.027136
7       0.020456
8       0.003856
9       1.000000
10      0.179278
11      0.021722
12      0.019123
13      0.031561
14      0.022215
15      0.001277
16      0.000075
17      0.149515
18      0.020193
19      0.051197
20      0.000887
21      0.288905
22      0.674759
23      0.019193
24      0.045913
25      0.041374
26      0.149109
27      0.930635
28      0.053291
29      0.000238
          ...   
8094    0.060765
8095    0.085090
8096    0.994013
8097    0.973971
8098    0.999793
8099    0.038781
8100    0.209793
8101    0.073587
8102    0.999999
8103    0.108171
8104    0.229282
8105    0.095100
8106    0.073526
8107    0.240781
8108    0.999997
8109    0.073800
8110    0.147432
8111    0.069788
8112    0.040924
8113    0.007011
8114    0.245022
8115    0.124109
8116    0.393421
8117    0.002838
8118    0.112165
8119    0.005708
8120    0.003021
8121    0.1301

On these scores , we will consider many cutoffs between 0 to 1

In [43]:
cutoffs=np.linspace(0,1,100)

For each of these cutoff , we are going to look at TP,FP,TN,FN values and caluclate KS. Then we'll chose the best cutoff as the one having highest KS.

In [45]:
KS_cut=[]
for cutoff in cutoffs:
    predicted=pd.Series([0]*len(y_train))
    predicted[prob_score>cutoff]=1
    df=pd.DataFrame(list(zip(y_train,predicted)),columns=["real","predicted"])
    TP=len(df[(df["real"]==1) &(df["predicted"]==1) ])
    FP=len(df[(df["real"]==0) &(df["predicted"]==1) ])
    TN=len(df[(df["real"]==0) &(df["predicted"]==0) ])
    FN=len(df[(df["real"]==1) &(df["predicted"]==0) ])
    P=TP+FN
    N=TN+FP
    KS=(TP/P)-(FP/N)
    KS_cut.append(KS)

cutoff_data=pd.DataFrame(list(zip(cutoffs,KS_cut)),columns=["cutoff","KS"])

KS_cutoff=cutoff_data[cutoff_data["KS"]==cutoff_data["KS"].max()]["cutoff"]

Now we'll see how this model with the cutoff determined here , performs on the test data.

In [46]:
KS_cutoff

44    0.444444
Name: cutoff, dtype: float64

In [47]:
# Performance on test data
prob_score_test=pd.Series(list(zip(*logr.predict_proba(x_test)))[1])

predicted_test=pd.Series([0]*len(y_test))
predicted_test[prob_score_test>float(KS_cutoff)]=1

df_test=pd.DataFrame(list(zip(y_test,predicted_test)),columns=["real","predicted"])

k=pd.crosstab(df_test['real'],df_test["predicted"])
print('confusion matrix :\n \n ',k)
TN=k.iloc[0,0]
TP=k.iloc[1,1]
FP=k.iloc[0,1]
FN=k.iloc[1,0]
P=TP+FN
N=TN+FP

confusion matrix :
 
  predicted     0    1
real                
0          1646  161
1            26  198


In [48]:
# Accuracy of test
(TP+TN)/(P+N)

0.9079271294928607

In [49]:
# Sensitivity on test
TP/P

0.8839285714285714

In [50]:
#Specificity on test
TN/N

0.9109020475926951

Next we see how cutoff determined by F_beta score performs on test data for beta values : 0.5,1,2

In [ ]:
cutoffs=np.linspace(0.010,0.99,100)
def Fbeta_perf(beta,cutoffs,y_train,prob_score):
    FB_cut=[]
    for cutoff in cutoffs:
        predicted=pd.Series([0]*len(y_train))
        predicted[prob_score>cutoff]=1
        df=pd.DataFrame(list(zip(y_train,predicted)),columns=["real","predicted"])

        TP=len(df[(df["real"]==1) &(df["predicted"]==1) ])
        FP=len(df[(df["real"]==0) &(df["predicted"]==1) ])
        FN=len(df[(df["real"]==1) &(df["predicted"]==0) ])
        P=TP+FN
        
        
        Precision=TP/(TP+FP)
        Recall=TP/P
        FB=(1+beta**2)*Precision*Recall/((beta**2)*Precision+Recall)
        FB_cut.append(FB)

    cutoff_data=pd.DataFrame(list(zip(cutoffs,FB_cut)),columns=["cutoff","FB"])

    FB_cutoff=cutoff_data[cutoff_data["FB"]==cutoff_data["FB"].max()]["cutoff"]

    prob_score_test=pd.Series(list(zip(*logr.predict_proba(x_test)))[1])

    predicted_test=pd.Series([0]*len(y_test))
    predicted_test[prob_score_test>float(FB_cutoff)]=1

    df_test=pd.DataFrame(list(zip(y_test,predicted_test)),columns=["real","predicted"])

    k=pd.crosstab(df_test['real'],df_test["predicted"])
#     print('confusion matrix :\n \n ',k)
    TN=k.iloc[0,0]
    TP=k.iloc[1,1]
    FP=k.iloc[0,1]
    FN=k.iloc[1,0]
    P=TP+FN
    N=TN+FP
    print('For beta :',beta)
    print('Accuracy is :',(TP+TN)/(P+N))
    print('Sensitivity is :',(TP/P))
    print('Specificity is :',(TN/N))
    print('\n \n \n')

In [ ]:
Fbeta_perf(0.5,cutoffs,y_train,prob_score)
Fbeta_perf(1,cutoffs,y_train,prob_score)
Fbeta_perf(2,cutoffs,y_train,prob_score)

You can see that low beta < 1 favors Specificity where as beta > 1 favors sensitivity

We'll conclude our discussion here. Please do the practice exercises . If you face any issue we'll discuss that either in class or QA forum on LMS.

Prepared By : Lalit Sachan (lalit.sachan@edvancer.in)

In case of any doubts or errata alert; please take to QA forum for discussion.

Doubts will be discussed in live class sessions too. [This doesnt apply for self paced students]